In [ ]:
%pip install optuna
%pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.6 MB/s eta 0:00:00


In [ ]:
'''General packages'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sklearn
import cv2
from PIL import Image
import os
import time
from tqdm import *

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import glob
from datetime import datetime
import argparse
import optuna
from optuna.trial import TrialState
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from facenet_pytorch import MTCNN, InceptionResnetV1

torch.manual_seed(42)


In [ ]:
! git clone https://github.com/shellyfra/Defend_against_attribute_attacks_in_face_recognition_models.git
from Defend_against_attribute_attacks_in_face_recognition_models.utils import train, imshow_no_normalization, set_parameter_requires_grad, get_params_to_update, eval_acc, imshow
%cd Defend_against_attribute_attacks_in_face_recognition_models/StarGAN_with_our_changes

Cloning into 'Defend_against_attribute_attacks_in_face_recognition_models'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 304 (delta 4), reused 6 (delta 2), pack-reused 294
Receiving objects: 100% (304/304), 21.03 MiB | 23.36 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/content/Defend_against_attribute_attacks_in_face_recognition_models/StarGAN_with_our_changes


## Load the CelebA-HQ dataset

In [ ]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ES-jbCNC6mNHhCyR4Nl1QpYBlxVOJ5YiVerhDpzmoS9ezA?download=1 -O CelebA_HQ_facial_identity_dataset.zip
!unzip CelebA_HQ_facial_identity_dataset.zip -d ./CelebA_HQ_facial_identity_dataset

Streaming output truncated to the last 5000 lines.
  inflating: ./CelebA_HQ_facial_identity_dataset/test/563/8709.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/563/9976.jpg  
   creating: ./CelebA_HQ_facial_identity_dataset/test/564/
  inflating: ./CelebA_HQ_facial_identity_dataset/test/564/6738.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/564/7152.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/564/739.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/564/7871.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/564/9481.jpg  
   creating: ./CelebA_HQ_facial_identity_dataset/test/571/
  inflating: ./CelebA_HQ_facial_identity_dataset/test/571/4864.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/571/5042.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/571/748.jpg  
  inflating: ./CelebA_HQ_facial_identity_dataset/test/571/9549.jpg  
   creating: ./CelebA_HQ_facial_identity_dataset/test/573/
  inflating: ./

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

transforms_test = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.

In this example, we optimize the validation accuracy of fashion product recognition using
PyTorch and FashionMNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole FashionMNIST dataset,
we here use a small subset of it.

"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DIR = os.getcwd()
num_epochs = 5

In [ ]:
faceNet = InceptionResnetV1(pretrained='vggface2')
faceNet = set_parameter_requires_grad(faceNet, 307) # number of identities = 307 in celebA-HQ subset
# new_resnet
params_to_update = []
for name,param in faceNet.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

  0%|          | 0.00/107M [00:00<?, ?B/s]

	 logits.weight
	 logits.bias


In [ ]:
def get_celeba(batch_size):
    data_dir = './CelebA_HQ_facial_identity_dataset'
    train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    print('Train dataset size:', len(train_dataset))
    print('Test dataset size:', len(val_dataset))

    return train_dataloader, val_dataloader, len(val_dataset)


def objective(trial):
    global faceNet
    global device
    # Generate the model.
    faceNet = faceNet.to(device)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 64])
    optimizer = getattr(optim, optimizer_name)(params_to_update, lr=lr)
    criterion = nn.CrossEntropyLoss()   
    num_epochs = 5
    # Get the FashionMNIST dataset.
    train_loader, valid_loader, len_val_dataset = get_celeba(batch_size)
    n_train_examples = batch_size * 30
    n_valid_examples = batch_size * 10
    # Training of the model.
    for epoch in range(num_epochs):
        faceNet.train()
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * batch_size >= n_train_examples:
                break
            inputs, labels = inputs.to(device), labels.to(device)
            # forward inputs and get output
            optimizer.zero_grad()
            outputs = faceNet(inputs)
            loss = criterion(outputs, labels)
            # get loss value and update the network weights
            loss.backward()
            optimizer.step()

        # Validation of the model.
        faceNet.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * batch_size >= n_valid_examples:
                    break
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = faceNet(inputs)
                # Get the index of the max log-probability.
                _, preds = torch.max(outputs, 1)
                correct += torch.sum(preds == labels.data)
                # print(f"batch acc = {torch.sum(preds == labels.data)/batch_size}")

        accuracy = (correct / min(len_val_dataset, n_valid_examples)) *100.
        print(f"accuracy : {accuracy}, epoch = {epoch}")
        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

    

In [ ]:
study = optuna.create_study(study_name="FaceNet", direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100, timeout=6000000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))


[I 2023-01-21 21:43:00,545] A new study created in memory with name: FaceNet


Train dataset size: 4263
Test dataset size: 1215
accuracy : 0.0, epoch = 0
accuracy : 5.0, epoch = 1
accuracy : 2.5, epoch = 2
accuracy : 1.25, epoch = 3


[I 2023-01-21 21:43:33,691] Trial 0 finished with value: 1.25 and parameters: {'optimizer': 'SGD', 'lr': 0.002467861496294836, 'batch_size': 8}. Best is trial 0 with value: 1.25.


accuracy : 1.25, epoch = 4
Train dataset size: 4263
Test dataset size: 1215
accuracy : 2.5, epoch = 0
accuracy : 2.5, epoch = 1
accuracy : 2.5, epoch = 2
accuracy : 2.5, epoch = 3


[I 2023-01-21 21:43:56,056] Trial 1 finished with value: 0.0 and parameters: {'optimizer': 'Adam', 'lr': 1.7858589263777402e-05, 'batch_size': 8}. Best is trial 0 with value: 1.25.


...

accuracy : 88.59375, epoch = 0
Train dataset size: 4263
Test dataset size: 1215


[I 2023-01-21 22:39:41,098] Trial 98 pruned. 


accuracy : 83.75, epoch = 0
Train dataset size: 4263
Test dataset size: 1215


[I 2023-01-21 22:39:45,590] Trial 99 pruned. 


accuracy : 91.25, epoch = 0
Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  66
  Number of complete trials:  34
Best trial:
  Value:  100.0
  Params: 
    optimizer: SGD
    lr: 0.013545514914041368
    batch_size: 8


In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_intermediate_values(study)
fig.show()

## Optuna on FaceNet with Augmentations

In [ ]:
# install our pre-trained models on celebA-HQ dataset (with / without augmentation and with attributes)
!bash download.sh models-weights

--2023-01-22 06:06:53--  https://docs.google.com/uc?export=download&confirm=t&id=1G5QNRswQjVNJ3gDTf0W8-vsiuFuVqDY_
Resolving docs.google.com (docs.google.com)... 74.125.199.100, 74.125.199.139, 74.125.199.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.199.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/76hsmgpssk7avurt5f05u6qcmn19341p/1674367575000/05560778185249539437/*/1G5QNRswQjVNJ3gDTf0W8-vsiuFuVqDY_?e=download&uuid=343a0b24-c3a2-4d04-8b9f-76b185fcdd21 [following]
--2023-01-22 06:06:53--  https://doc-14-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/76hsmgpssk7avurt5f05u6qcmn19341p/1674367575000/05560778185249539437/*/1G5QNRswQjVNJ3gDTf0W8-vsiuFuVqDY_?e=download&uuid=343a0b24-c3a2-4d04-8b9f-76b185fcdd21
Resolving doc-14-08-docs.googleusercontent.com (doc-14-08-docs.googleusercontent.com)... 74.125.142.132, 

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return torch.clip(tensor + torch.randn(tensor.size()) * self.std + self.mean, 0, 1)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

transforms_attack2 = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # Normalization is done in dataloader
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    AddGaussianNoise(0., 0.1)
    ])

transforms_attack3 = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # Normalization is done in dataloader
    transforms.ColorJitter(brightness=.1, hue=.1),
    AddGaussianNoise(0., 0.1)
    ])

transforms_orig = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    ])

In [ ]:
# img_path = '/content/Defend_against_attribute_attacks_in_face_recognition_models/StarGAN_with_our_changes/'

In [ ]:
for filepath in glob.glob("./CelebA_HQ_facial_identity_dataset/train/*/"):
    for img_path in glob.glob(filepath + '/*'):
      test_img = Image.open(img_path).convert('RGB')
      test_img_attack2 = transforms_attack2(test_img)
      test_img_attack3 = transforms_attack3(test_img)
      torchvision.utils.save_image(test_img_attack2, img_path.split('.jpg')[0] + '_1.jpg')
      torchvision.utils.save_image(test_img_attack3,  img_path.split('.jpg')[0] + '_2.jpg')


In [ ]:
for filepath in glob.glob("./CelebA_HQ_facial_identity_dataset/test/*/"):
    for img_path in glob.glob(filepath + '/*'):
      test_img = Image.open(img_path).convert('RGB')
      test_img_attack2 = transforms_attack2(test_img)
      test_img_attack3 = transforms_attack3(test_img)
      torchvision.utils.save_image(test_img_attack2, img_path.split('.jpg')[0] + '_1.jpg')
      torchvision.utils.save_image(test_img_attack3,  img_path.split('.jpg')[0] + '_2.jpg')

In [ ]:
f'{DIR}/models/{model_name}'

'/content/Defend_against_attribute_attacks_in_face_recognition_models/StarGAN_with_our_changes/models/CelebA_HQ_Facenet_without_aug.pth'

In [ ]:
model_name = 'CelebA_HQ_Facenet_without_aug.pth'
faceNet = InceptionResnetV1(pretrained='vggface2')
faceNet = set_parameter_requires_grad(faceNet, 307) # number of identities = 307 in celebA-HQ subset

state = torch.load(f'{DIR}/models/{model_name}', map_location=device)
faceNet.load_state_dict(state['net'])

params_to_update = []
for name,param in faceNet.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 logits.weight
	 logits.bias


In [ ]:
study = optuna.create_study(study_name="FaceNet_fine_tune", direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=100, timeout=6000000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))


[I 2023-01-22 06:21:04,564] A new study created in memory with name: FaceNet_fine_tune


Train dataset size: 12789
Test dataset size: 3645
accuracy : 83.75, epoch = 0
accuracy : 90.0, epoch = 1
accuracy : 76.25, epoch = 2
accuracy : 86.25, epoch = 3


[I 2023-01-22 06:21:22,663] Trial 0 finished with value: 87.5 and parameters: {'optimizer': 'SGD', 'lr': 9.71437253200813e-05, 'batch_size': 8}. Best is trial 0 with value: 87.5.


accuracy : 87.5, epoch = 4
Train dataset size: 12789
Test dataset size: 3645
accuracy : 87.5, epoch = 0
accuracy : 78.75, epoch = 1
accuracy : 68.75, epoch = 2
accuracy : 75.0, epoch = 3


[I 2023-01-22 06:21:33,427] Trial 1 finished with value: 85.0 and parameters: {'optimizer': 'SGD', 'lr': 6.29475643086052e-05, 'batch_size': 8}. Best is trial 0 with value: 87.5.


...

accuracy : 59.531253814697266, epoch = 0
Train dataset size: 12789
Test dataset size: 3645


[I 2023-01-22 06:32:35,853] Trial 98 pruned. 


accuracy : 51.25, epoch = 0
Train dataset size: 12789
Test dataset size: 3645


[I 2023-01-22 06:32:38,129] Trial 99 pruned. 


accuracy : 47.5, epoch = 0
Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  94
  Number of complete trials:  6
Best trial:
  Value:  89.21875
  Params: 
    optimizer: SGD
    lr: 0.000615545095147844
    batch_size: 64


In [ ]:
"""
Best trial:
  Value:  89.21875
  Params: 
    optimizer: SGD
    lr: 0.000615545095147844
    batch_size: 64
"""

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_intermediate_values(study)
fig.show()